In [1]:

#%% Importing modules and data
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
# import pandas.stats.moments as st
import statsmodels.api as sm # import statsmodels 
from pandas import ExcelWriter
import matplotlib.pyplot as pyplot
import scipy.stats as st
import os
import quandl as qd
from collections import defaultdict
import seaborn as sns
%matplotlib inline

def save_xls(list_dfs, xls_path,sheet_names):
    writer = ExcelWriter(xls_path)
    for n, df in enumerate(list_dfs):
        df.to_excel(writer, sheet_names[n])
    writer.save()
    return

C:\Users\xexx\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Importing data from Quandl

btc = qd.get("BITFINEX/BTCUSD", authtoken="-ZxrLoLy3vKgLtLraaMn")
btc.columns = ['High','Low','Mid','Close','Bid','Ask','Volume']

In [32]:
def ichimoku_cloud(df,w9,w26,w52):
    #crypto settings are 20, 60, 120, 30 see Josh video for confirmation
    
    # Tenkan-sen (Conversion Line): (9-period high + 9-period low)/2))
    df['period9_high'] = pd.Series.rolling(df['High'], w9).max()
    df['period9_low'] = pd.Series.rolling(df['Low'], w9).min()
    df['tenkan_sen'] = (df['period9_high'] + df['period9_low']) / 2
    
    # Kijun-sen (Base Line): (26-period high + 26-period low)/2))
    df['period26_high'] = pd.Series.rolling(df['High'], w26).max()
    df['period26_low'] = pd.Series.rolling(df['Low'], w26).min()
    df['kijun_sen'] = (df['period26_high'] + df['period26_low']) / 2
    
    df['period52_high'] = pd.Series.rolling(df['High'], w52).max()
    df['period52_low'] = pd.Series.rolling(df['Low'], w52).min()
    df['senkou_span_b'] = (df['period52_high'] + df['period52_low'] ) /2 
    
    df ['chikou_span'] = df['Close'].shift(22) 
    return df


In [33]:
ichimoku_cloud(btc,9,26,52)

,High,Low,Mid,Close,Bid,Ask,Volume,return,period9_high,period9_low,tenkan_sen,period26_high,period26_low,kijun_sen,period52_high,period52_low,senkou_span_b,chikou_span
Date,,,,,,,,,,,,,,,,,,
2014-04-15,513.9000,452.000000,504.235000,505.000000,503.500000,504.970000,21013.584774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-04-16,547.0000,495.000000,537.500000,538.000000,537.000000,538.000000,29633.358705,0.065347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-04-17,538.5000,486.100000,507.020000,508.000000,506.040000,508.000000,20709.783819,-0.055762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-04-18,509.0000,474.250000,483.770000,482.750000,482.750000,484.790000,10458.045243,-0.049705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-04-19,513.9899,473.830000,505.010650,507.499900,502.531300,507.490000,8963.618369,0.051269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-04-20,517.9950,492.200000,500.745753,501.440000,500.071506,501.420000,4921.588803,-0.011941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-04-21,515.6460,485.000000,497.090000,497.000000,497.000000,497.180000,8132.144453,-0.008854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-04-22,506.0000,488.800000,491.300003,491.200000,491.209991,491.390015,3105.516105,-0.011670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-04-23,496.0000,482.880000,490.819500,490.030000,490.040000,491.599000,4716.703433,-0.002382,547.0000,452.00,499.50000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
